In [ ]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
import pandas as pd
from bagpy import bagreader

# import data from recording at Klitten FVZ
results_file = "./Klitten/loc_results_2023-11-01-12-53-52_0.bag"
sensor_file = "./Klitten/recordedData_sensors_locpc_2023-11-01-12-53-52_0.bag"

result_reader = bagreader(results_file)
sensor_reader = bagreader(sensor_file)


# Thius function extracts the sensordata into CSV-files

map_path = result_reader.message_by_topic(topic="/odometry/filtered/map")
#odom_path = result_reader.message_by_topic(topic="/odometry/filtered/odom")
#gps_odometry_path  = result_reader.message_by_topic(topic="/odometry/gps")
imu_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/imu")
#speed_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/speed")
#steeringAngle_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/steeringAngle")
#gnss_oem = sensor_reader.message_by_topic(topic="/localization_pc/can0/decoded/gnss")
#imu_anavs = sensor_reader.message_by_topic(topic="/localization_pc/sensor/anavs/imu")
#navsat_anavs = sensor_reader.message_by_topic(topic="/localization_pc/sensor/anavs/navsat")


# You can then use Pandas to read the files.
df_map_path = pd.read_csv(map_path)  
#df_odom_path = pd.read_csv(odom_path)  
# df_gps_path = pd.read_csv(gps_odometry_path)
#df_speed = pd.read_csv(speed_oem)
df_imu = pd.read_csv(imu_oem)

# 我写的
# df_imu_anavs = pd.read_csv(imu_anavs)
# df_navsat_anvas = pd.read_csv(navsat_anavs)

#### Die Zeitdaten von `header.stamp.secs` sollen verschoben werden, sodass sie ab 0 Sekunden beginnen.

In [ ]:
df_map_path['header.stamp.secs'] = df_map_path['header.stamp.secs'] - df_map_path['header.stamp.secs'].iloc[0]
df_imu['header.stamp.secs'] = df_imu['header.stamp.secs'] - df_imu['header.stamp.secs'].iloc[0]

#### Die Anzeigeformatierung für DataFrame einstellen

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 20)

#### Eine neue Spalte `secs.nsecs` zwischen `header.stamp.secs` und `header.stamp.nsecs` einfügen, sodass der Zeitstempel in Sekunden mit Dezimalstellen dargestellt wird.

In [ ]:
# 计算新列
df_imu['secs.nsecs'] = df_imu['header.stamp.secs'] + df_imu['header.stamp.nsecs'] * 1e-9
df_imu['secs.nsecs'] = df_imu['secs.nsecs'].round(9)
# 调整列的位置，将 'secs.nsecs' 放到 'header.stamp.nsecs' 后面
columns = df_imu.columns.tolist()
index = columns.index('header.stamp.nsecs') + 1  # 获取目标列的索引并加1
columns.insert(index, columns.pop(columns.index('secs.nsecs')))  # 移动 secs.nsecs 列

# 重新排列列顺序
df_imu = df_imu[columns]

In [ ]:
# 计算新列
df_map_path['secs.nsecs'] = df_map_path['header.stamp.secs'] + df_map_path['header.stamp.nsecs'] * 1e-9
df_map_path['secs.nsecs'] = df_map_path['secs.nsecs'].round(9)
# 调整列的位置，将 'secs.nsecs' 放到 'header.stamp.nsecs' 后面
columns = df_map_path.columns.tolist()
index = columns.index('header.stamp.nsecs') + 1  # 获取目标列的索引并加1
columns.insert(index, columns.pop(columns.index('secs.nsecs')))  # 移动 secs.nsecs 列

# 重新排列列顺序
df_map_path = df_map_path[columns]

#### Die für die control-input \( u \) erforderlichen Daten aus den Sensordaten extrahieren.

In [ ]:
# u, cov_u = func_u(k, dt_step)
import ast
# 定义函数 func_u
def func_u(k, dt_step):
    t = k * dt_step

    # 从 df_map_path 中找到与 t 值最接近的 index
    idx_v               = (df_map_path['secs.nsecs'] - t).abs().idxmin()
    v                   = df_map_path.loc[idx_v, 'twist.twist.linear.x']
    cov_v_str           = df_map_path.loc[idx_v, 'twist.covariance']
    cov_v_parsed_matrix = ast.literal_eval(cov_v_str)
    cov_v_matrix        = np.array(cov_v_parsed_matrix ).reshape(6, 6)
    cov_v               = cov_v_matrix[0, 0]
    
    # 从 df_imu 中找到与 t 值最接近的 index
    idx_omega = (df_imu['secs.nsecs'] - t).abs().idxmin()
    omega     = df_imu.loc[idx_omega, 'angular_velocity.z']
    cov_omega = df_imu.loc[idx_omega, 'angular_velocity_covariance_8']
    
    u = np.array([v, omega])
    cov_u = np.array([cov_v, cov_omega])
    return u, cov_u
    
k = 200
dt_step = 0.1
u, cov_u = func_u(k, dt_step)

#### Die für die Zustansmatrix (X) erforderlichen Daten aus den Sensordaten extrahieren.

In [ ]:
# z, cov_z = func_z(k, dt_step)   
import ast
from sympy import symbols, Matrix

# 定义函数 func_z
def func_z(k, dt_step):
    t = k * dt_step

    # 从 df_map_path 中找到与 t 值最接近的 index
    idx                    = (df_map_path['secs.nsecs'] - t).abs().idxmin()
    x                      = df_map_path.loc[idx, 'pose.pose.position.x']
    y                      = df_map_path.loc[idx, 'pose.pose.position.y']
    theta                  = df_map_path.loc[idx, 'pose.pose.orientation.z']
    z                      = Matrix([x, y, theta])
    
    cov_pose_str           = df_map_path.loc[idx, 'pose.covariance']
    cov_pose_parsed_matrix = ast.literal_eval(cov_pose_str)
    cov_x                  = cov_pose_parsed_matrix[0]
    cov_y                  = cov_pose_parsed_matrix[7]
    cov_xy                 = cov_pose_parsed_matrix[1]
    cov_theta              = cov_pose_parsed_matrix[14]
    cov_z                  = np.array([cov_x, cov_y, cov_xy, cov_theta])
    
    # cov_pose_str           = df_map_path.loc[idx, 'pose.covariance']
    # cov_pose_parsed_matrix = ast.literal_eval(cov_pose_str)
    # cov_pose_matrix        = np.array(cov_pose_parsed_matrix).reshape(6, 6)
    # cov_x                  = cov_pose_matrix[0, 0]
    # cov_y                  = cov_pose_matrix[1, 1]
    # cov_xy                 = cov_pose_matrix[1, 0]
    # cov_theta              = cov_pose_matrix[2, 2]
    # cov_z                  = np.array([cov_x, cov_y, cov_xy, cov_theta])

    return z, cov_z
    
k = 10000
dt_step = 0.1
z, cov_z = func_z(k, dt_step)   

## Determine the required measurement dataset

In [ ]:
def SensorData(sensor_type):
    """
    Function to retrieve specific columns from the appropriate DataFrame based on input parameters..

    Parameters: One of the following strings - 'x', 'y', 'heading_angle', 'speed', 'yaw_rate'.

    Returns:
        pd.Series: The corresponding column from the appropriate DataFrame.
    """
    # Define the mapping between sensor types and DataFrame columns
    column_mapping = {
        "x":             ("df_map_path", "pose.pose.position.x"),
        "y":             ("df_map_path", "pose.pose.position.y"),
        "heading_angle": ("df_map_path", "pose.pose.orientation.z"),
        "speed":         ("df_map_path", "twist.twist.linear.x"),
        "yaw_rate":      ("df_imu",      "angular_velocity.z"),
    }

    # Check if the input sensor_type is valid
    if sensor_type not in column_mapping:
        raise ValueError(f"Invalid sensor type: {sensor_type}. Must be one of {list(column_mapping.keys())}")

    # Get the DataFrame and column name
    df_name, column_name = column_mapping[sensor_type]

    # Retrieve the DataFrame object dynamically
    df = globals().get(df_name)
    if df is None:
        raise ValueError(f"DataFrame {df_name} is not defined in the current environment.")

    # Return the corresponding column from the appropriate DataFrame
    return df[column_name]

# Example usage:
# result = SensorData("heading_angle")
# print(result)


In [ ]:
result = SensorData("heading_angle")
result

# 1. Design the State Variables

$$
\mathbf{X} =
\begin{bmatrix}
x \\
y \\
\theta \\
v \\
\omega
\end{bmatrix}
$$

- **[x, y]**: Position of the vehicle  
- **θ\theta**: Heading angle (\(^\circ\))  
- **v**: Speed (\([m/s]\))  
- **ω**: Yaw rate (\([rad/s]\))

# 2. Design the System Model 

## 2.1 System Jacobian Matrix J

In [ ]:
from sympy import symbols, Matrix, sin, cos, init_printing

# initialize to show in the form of LaTex
init_printing(use_latex="mathjax", fontsize="16pt")

# define the symbles of variables
x, y, theta, v, omega, time = symbols('x y theta v omega T')  

# define the state vector
X = Matrix([x, y, theta])  

# define state transition f(x, u)
f_xu = Matrix([
    x + v/omega * (sin(theta + omega*time) - sin(theta)), 
    y + v/omega * (cos(theta) - cos(theta + omega*time)),  
    theta + omega*time
])

# calculate jacobian matrix J
J = f_xu.jacobian(X)

J

In [ ]:
# reduce common expressions
# R = symbols('R')
# J = J.subs(v/omega, R)  # 这里暂且放一下，不对公式进行简化
# J

## 2.2 control input Noise M and Jacobian Matrix V 

In [ ]:
u = Matrix([v, omega])
V = f_xu.jacobian(u)
# V = V.subs(v/omega, "R")
V

# 3. Design the Measurement Model

In [ ]:
# Die Sensoren können direkt die Messdaten der Variablen liefern
z = Matrix([x, y, theta])
H = z.jacobian(X)
H

In [ ]:
print(type(H))

calculate measurement from system state

In [ ]:
from numpy import array

def H_x(X):
    """convert prediction to measurement"""
    H_x = array([
        [X[0, 0]],
        [X[1, 0]],
        [X[2, 0]]
    ])
    return H_x

# 4. Design Measurement Noise

# 5. Implementation

In [ ]:
from filterpy.kalman import ExtendedKalmanFilter as EKF
from numpy import array, sqrt
class CTRV_EKF(EKF):
    def __init__(self):  # dt is value for specific timestep
        # state dimension 3, measurement dimension 3, control input dimension 2
        EKF.__init__(self, 3, 3, 2)
        
        # symble definition
        x, y, theta, v, omega, time = symbols('x y theta v omega T')  # "T" is symble variable for symble expression, "time" is the saved reference-name of "T"
        # "="前面的x, y 是通过 sympy.symbols('x y ...') 定义的符号变量; 这些符号变量是用来进行符号运算（如雅可比矩阵计算）的数学符号; 这些符号变量会被赋值到 self.x_x 和 self.x_y
        # "="后面是 sympy.symbols 定义的字符串 'x y'，仅仅是符号定义的文本，不涉及变量引用
        
        # nonlinear system model f(x, u)    
        self.f_xu = Matrix([
            x + v/omega * (sin(theta + omega*time) - sin(theta)), 
            y + v/omega * (cos(theta) - cos(theta + omega*time)),  
            theta + omega*time
        ])

        # calculate jacobian matrix
        self.F_J = self.f_xu.jacobian(Matrix([x, y, theta]))
        self.V_J = self.f_xu.jacobian(Matrix([v, omega]))

        # replace dictionary and  it's variables
        self.subs                      = {x: 0, y: 0, theta: 0, v: 0, omega: 0, time: dt}  # 等号后面的x, y 是替换字典 self.subs 的键。
        self.x_x, self.x_y,            = x, y  # 引用的是symble definition中的符号变量 x 和 y

    
    def predict(self, u, cov_v, cov_omega):  # cov_v, cov_omega由传感器直接提供
        self.X         = self.driving(self.X, u, self.dt)  # self.X after "="：the current state vector; self.X before "="：the next state vector
        self.cov_v     = cov_v
        self.cov_omega = cov_omega 
        
        # 下面这段代码将状态向量 self.x 和控制输入 u 的值绑定到符号变量中，便于后续将符号表达式替换为具体数值。
        self.subs[self.x_x]   = self.X[0, 0]  # self.X is 2-D Matrix，这行代码将符号变量 self.x_x 绑定到 self.x[0, 0] 的具体值。
        self.subs[self.x_y]   = self.X[1, 0]
        self.subs[self.theta] = self.x[2, 0]        
        self.subs[self.v]     = u[0]          # u is 1-D Array
        self.subs[self.omega] = u[1]
        
        # predict the value of Jacobian Matrix
        J = array(self.F_J.evalf(subs=self.subs)).astype(float)
        V = array(self.V_j.evalf(subs=self.subs)).astype(float)
        
        # Noise covariance Matrix of control input
        M = array([[self.cov_v, 0], 
                   [0, self.cov_omega]])

        # Covariance Matrix of Prediction 
        self.P = F @ self.P @ F.T + V @ M @ V.T  # F：状态转移矩阵（雅可比矩阵), V：控制输入对状态影响的雅可比矩阵 ∂f/∂u, M：控制输入的噪声协方差矩阵
                                                 # 在 Python 中，@ 是矩阵乘法运算符，用于执行矩阵与矩阵或矩阵与向量的乘法运算。它是在 Python 3.5 中引入的，用于增强矩阵运算的可读性，与 NumPy 的 np.dot() 功能等效，但语法更加直观。                                        

    
    def driving(self, X, u, dt):
        heading_angle = X[2, 0]
        velocity      = u[0]
        yaw_rate      = u[1]
        
        radius        = velocity/yaw_rate  # R = v / ω
        beta          = yaw_rate * dt
        
        dX = np.array([
            [radius * (sin(heading_angle + beta) - sin(heading_angle))],
            [radius * (cos(heading_angle) - cos(heading_angle + beta))],
            [beta]
        ])

        return X + dX
    

In [ ]:
# Roger R Labbe Jr. Kalman_and_Bayesian_Filters_in_Python.
def residual(a, b): 
    """ compute residual (a-b) between measurements containing 
    [position, heading_angle]. heading_angle is normalized to [-pi, pi)"""
    y = a - b
    y[2] = y[2] % (2 * np.pi)    # force in range [0, 2 pi)
    if y[2] > np.pi:             # move to [-pi, pi)
        y[2] -= 2 * np.pi
    return y

In [ ]:
from filterpy.stats import plot_covariance_ellipse
from math import sqrt, tan, cos, sin, atan2
import matplotlib.pyplot as plt

def ekf_update(ekf, z):
    ekf.update(z, 
               HJacobian = H,                                         # 观测模型的雅可比矩阵 H 
               Hx        = H_x,                                       # 测量函数 h(x)，直接将状态变量转换为测量值
               residual  = residual)

def run_CTRV(dt_step      = 1.0,                                      # secs, dt不可以直接指定固定值，而是要根据测量数据的时间戳进行决定，或者确定一个dt，截取传感器数据中与之时刻最接近的数据
             dt_track     = 0.1,
             sim_duration = 20,                                       # secs
             ylim         = None):
    
    ekf          = CTRV_EKF(dt, cov_v=cov_v, cov_omega=cov_omega)
    ekf.X        = array([[2, 6, .3]]).T                              # 这里需要与传感器测量数据链接，然后确定初始值
    ekf.P        = np.diag([.1, .1, .1])                              # 这里和ekf.X一样需要先给定一个初始值，然后在ekf运行的怕过程中会自动更新，这个初始值是什么有待商榷
    sim_X        = ekf.X.copy()                                       # simulated position and heading_angle
    u            = u_0                                                # u_0也需要斟酌一下为何值
    track        = []                                                 # 用于记录汽车的运行轨迹
    k            = 0                                                  # timestep for prediction and update
    step         = int(dt_step / dt_track)                            
    ellipse_step = 2 * step
    track_step   = int(sim_duration / dt_track)
    
    for i in range(track_step):     
        sim_X = ekf.driving(sim_X, u, dt_track)                       # simulate the driving model of the car
        track.append(sim_X)                                           # 这里并不多余，因为要绘制汽车运行的轨迹图，当 ω ≠ 0 时，两次更新点之间的轨迹并不是一条直线，所以需要适当的增加一些轨迹点使轨迹图看起来平滑
    
        if i % step == 0:
            k         = k + 1
            u, cov_u  = func_u(k, dt_step)                            # func_u是用来获取在每次更新阶段时刻k所需的最新控制矩阵 u，现在还每写，等ekf框架搭建完成后再设计  
            cov_v     = cov_u[0]
            cov_omega = cov_u[1]
            ekf.predict(u, cov_v, cov_omega)                          # 函数调用需要在测量噪声矩阵的重新定义之后，因为ekf的R参数被重置了，cov_v, cov_omega用于计算M矩阵
                                                                      # ekf.predict执行后会产生新的 P 矩阵
            if i % ellipse_step == 0:
                plot_covariance_ellipse(
                    (ekf.X[0,0], ekf.X[1,0]), ekf.P[0:2, 0:2], 
                     std=6, facecolor='k', alpha=0.3)                 # 设置predict阶段的协方差椭圆的颜色为黑色，透明度为 0.3（范围是 0 到 1）
           
            z, cov_z          = func_z(k, dt_step)                    # func_z用来获取测量值和每次更新阶段时刻k所需的最新状态矩阵 X 的噪声协方差
            cov_x             = cov_z[0]
            cov_y             = cov_z[1]
            cov_xy            = cov_z[2]
            cov_theta         = cov_z[3]
            
            # Measurement Noise Matrix，在不重新定义整个 EKF 实例 的情况下，动态更新测量噪声协方差矩阵 R。直接在循环中动态修改 ekf.R 即可，而无需重新实例化扩展卡尔曼滤波器。这种方式既高效又灵活。
            diagonal_elements = [cov_x, cov_y, cov_theta]             # 对角元素
            ekf.R             = np.diag(diagonal_elements)
            efk.R[0, 1]       = cov_xy
            efk.R[1, 0]       = cov_xy
            
            ekf_update(ekf, z)
            
            if i % ellipse_step == 0:
                plot_covariance_ellipse(
                    (ekf.x[0,0], ekf.x[1,0]), ekf.P[0:2, 0:2],
                    std=6, facecolor='g', alpha=0.8)                  # 设置update阶段的协方差椭圆的颜色为绿色，透明度为 0.8（范围是 0 到 1）

    track = np.array(track)
    plt.plot(track[:, 0], track[:,1], color='k', lw=2)                # LineWidth = 2
    plt.axis('equal')
    plt.title("Klitten EKF")
    if ylim is not None: plt.ylim(*ylim)
    plt.show()
    return ekf  
    

In [ ]:
ekf = run_CTRV()

In [ ]:
from numpy import eye
H = eye(3)
Hx = Matrix([x,y,theta])

In [ ]:
H

In [ ]:
Hx